In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

In [2]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()

word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# Model Description
model_name = 'model-aw-1-multigpu-3'
model_dir = '../output/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

with open('/data/aviraj/dataset/sampled_train_data','rb') as f:
    train_data=pickle.load(f)
    
with open('/data/aviraj/dataset/val_data','rb') as f:
    val_data=pickle.load(f)
    

with open('/data/aviraj/dataset/fulldata_vocab_sense','rb') as f:
    vocab_sense=pickle.load(f)

    
sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2sense = dict((i, s) for i,s in enumerate(vocab))

len(vocab_sense)

# Parameters
mode = 'train'
num_senses = len(vocab_sense)
batch_size = 32
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 400
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.01
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True
moving_avg_deacy = 0.999
num_gpus = 8

In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# MODEL
device_num = 0
tower_grads = []
losses = []
predictions = []

x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
sense_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='sense_mask')
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
        if gpu_idx>3:
            device_num = 1
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % device_num):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), 1)

            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)

            def attention(input_x, input_mask, W_att):
                h_masked = tf.boolean_mask(input_x, input_mask)
                h_tanh = tf.tanh(h_masked)
                u = tf.matmul(h_tanh, W_att)
                a = tf.nn.softmax(u)
                c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
                return c

            with tf.variable_scope("attention"):
                W_att = tf.get_variable("W_att", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                c = tf.expand_dims(attention(h[0], x_mask[gpu_idx][0], W_att), 0)
                for i in range(1, batch_size):
                    c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[gpu_idx][i], W_att), 0)], 0)
                
                cc = tf.expand_dims(c, 1)
                c_final = tf.tile(cc, [1, max_sent_size, 1])
                h_final = tf.concat([c_final, h],2)
                flat_h_final = tf.reshape(h_final, [-1, 4*hidden_size])
           
            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[4*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_flat_h_final = tf.nn.dropout(flat_h_final, input_keep_prob)
                flat_logits_sense = tf.matmul(drop_flat_h_final, W) + b
                logits = tf.reshape(flat_logits_sense, [batch_size, max_sent_size, num_senses])
                predictions.append(tf.arg_max(logits, 2))
    

            float_sense_mask = tf.cast(sense_mask[gpu_idx], 'float')
 
            loss = tf.contrib.seq2seq.sequence_loss(logits, y[gpu_idx], float_sense_mask, name="loss")
            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = loss + l2_loss
            tf.summary.scalar("loss_{}".format(gpu_idx), total_loss)

            summaries = tf.get_collection(tf.GraphKeys.SUMMARIES, scope)

            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

tower_grads = average_gradients(tower_grads)
losses = tf.add_n(losses)/len(losses)
apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
summaries.append(tf.summary.scalar('total_loss', losses))
summaries.append(tf.summary.scalar('learning_rate', learning_rate))

for var in tf.trainable_variables():
    summaries.append(tf.summary.histogram(var.op.name, var))

variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())

train_op = tf.group(apply_grad_op, variables_averages_op)
saver = tf.train.Saver(tf.global_variables())
summary = tf.summary.merge(summaries)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [6]:
def model(xx, yy, mask, smask, train_cond=True):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    preds_sense = []
    true_sense = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size, -1])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        smask_re = smask[s:e].reshape([num_gpus, batch_size, -1])
 
        feed_dict = {x:xx_re, y:yy_re, x_mask:mask_re, sense_mask:smask_re, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
            
            if((j+1)%100==0):
                print("Steps: {}".format(step), ", Loss: {0:.4f}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            for i in range(num_gpus):
                preds_sense.append(pred[i][smask_re[i]])
                true_sense.append(yy_re[i][smask_re[i]])
        

        _losses +=_loss

    if(train_cond==False): 
        sense_preds = []
        sense_true = []
        for preds in preds_sense:
            for ps in preds:      
                sense_preds.append(ps)
                
        for trues in true_sense:
            for ts in trues:
                sense_true.append(ts)
        return _losses/num_batches, sense_preds, sense_true 

    return _losses/num_batches, step

def eval_score(yy, pred):
    f1 = f1_score(yy, pred, average='macro')
    accu = accuracy_score(yy, pred)
    return f1*100, accu*100

In [7]:
x_id_train, mask_train, sense_mask_train, y_train = train_data
x_id_val, mask_val, sense_mask_val, y_val = val_data

In [8]:
num_epochs = 10
log_period = 2

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, mask_train, sense_mask_train)
    time_taken = time.time() - start_time
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%log_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true = model(x_id_val, y_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_= eval_score(val_true, val_pred)
        time_taken = time.time() - start_time
        print("Val: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.4f}".format(time_taken))
    

Steps: 100 , Loss: 3.4693
Steps: 200 , Loss: 3.3920
Steps: 300 , Loss: 3.4569
Steps: 400 , Loss: 3.3758
Steps: 500 , Loss: 3.5039
Steps: 600 , Loss: 3.3876
Steps: 700 , Loss: 3.3381
Steps: 800 , Loss: 3.3603
Steps: 900 , Loss: 3.3070
Steps: 1000 , Loss: 3.3054
Steps: 1100 , Loss: 3.2467
Steps: 1200 , Loss: 3.3086
Steps: 1300 , Loss: 3.2269
Steps: 1400 , Loss: 3.1676
Steps: 1500 , Loss: 3.2810
Steps: 1600 , Loss: 3.1594
Steps: 1700 , Loss: 3.0882
Steps: 1800 , Loss: 3.0730
Steps: 1900 , Loss: 3.0617
Steps: 2000 , Loss: 2.9590
Steps: 2100 , Loss: 3.0731
Steps: 2200 , Loss: 3.0568
Steps: 2300 , Loss: 3.1115
Steps: 2400 , Loss: 2.9864
Steps: 2500 , Loss: 2.8796
Steps: 2600 , Loss: 2.9025
Steps: 2700 , Loss: 2.7012
Steps: 2800 , Loss: 2.8151
Epoch: 1 , Step: 2895 , loss: 3.6949 , Time: 18433.2341
Model Saved
Steps: 2995 , Loss: 2.7846
Steps: 3095 , Loss: 2.6822
Steps: 3195 , Loss: 2.6685
Steps: 3295 , Loss: 2.6697
Steps: 3395 , Loss: 2.7071
Steps: 3495 , Loss: 2.6359
Steps: 3595 , Loss: 2.6

Steps: 27455 , Loss: 1.6257
Steps: 27555 , Loss: 1.5045
Steps: 27655 , Loss: 1.5321
Steps: 27755 , Loss: 1.5782
Steps: 27855 , Loss: 1.7140
Steps: 27955 , Loss: 1.6255
Steps: 28055 , Loss: 1.5693
Steps: 28155 , Loss: 1.6484
Steps: 28255 , Loss: 1.5314
Steps: 28355 , Loss: 1.7482
Steps: 28455 , Loss: 1.7250
Steps: 28555 , Loss: 1.7087
Steps: 28655 , Loss: 1.5378
Steps: 28755 , Loss: 1.5513
Steps: 28855 , Loss: 1.4443
Epoch: 10 , Step: 28950 , loss: 1.6141 , Time: 18338.7458
Model Saved
Val: F1 Score:45.6626 Accuracy:51.5092 Loss:1.5524 , Time: 3248.9304


In [9]:
start_time = time.time()
train_loss, train_pred, train_true = model(x_id_train, y_train, mask_train, sense_mask_train, train_cond=False)        
f1_, accu_= eval_score(train_true, train_pred)
time_taken = time.time() - start_time
print("train: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))

train: F1 Score:47.4870 Accuracy:50.6842 Loss:1.5968 , Time: 14161.3469


In [ ]:
num_epochs = 10
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, mask_train, sense_mask_train)
    time_taken = time.time() - start_time
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%log_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true = model(x_id_val, y_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_= eval_score(val_true, val_pred)
        time_taken = time.time() - start_time
        print("Val: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.4f}".format(time_taken))
    

Steps: 29050 , Loss: 1.4071
Steps: 29150 , Loss: 1.4448
Steps: 29250 , Loss: 1.5366
Steps: 29350 , Loss: 1.5044
Steps: 29450 , Loss: 1.6531
Steps: 29550 , Loss: 1.7323
Steps: 29650 , Loss: 1.6403
Steps: 29750 , Loss: 1.7144
Steps: 29850 , Loss: 1.5374
Steps: 29950 , Loss: 1.4619
Steps: 30050 , Loss: 1.4569
Steps: 30150 , Loss: 1.5633
Steps: 30250 , Loss: 1.5508
Steps: 30350 , Loss: 1.7684
Steps: 30450 , Loss: 1.8366
Steps: 30550 , Loss: 1.5934
Steps: 30650 , Loss: 1.7341
Steps: 30750 , Loss: 1.5739
Steps: 30850 , Loss: 1.4988
Steps: 30950 , Loss: 1.5218
Steps: 31050 , Loss: 1.4916
Steps: 31150 , Loss: 1.6282
Steps: 31250 , Loss: 1.6183
Steps: 31350 , Loss: 1.5609
Steps: 31450 , Loss: 1.6294
Steps: 31550 , Loss: 1.6493
Steps: 31650 , Loss: 1.5855
Steps: 31750 , Loss: 1.6821
Epoch: 1 , Step: 31845 , loss: 1.5880 , Time: 18410.1649
Model Saved
Steps: 31945 , Loss: 1.5771
Steps: 32045 , Loss: 1.5847
Steps: 32145 , Loss: 1.5490
Steps: 32245 , Loss: 1.5886
Steps: 32345 , Loss: 1.5347
Steps: 

In [9]:
start_time = time.time()
val_loss, val_pred, val_true = model(x_id_val, y_val, mask_val, sense_mask_val, train_cond=False)        
f1_, accu_= eval_score(val_true, val_pred)
time_taken = time.time() - start_time
print("Val: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.4f}".format(time_taken))

Val: F1 Score:50.6887 Accuracy:56.7317 Loss:1.3617 , Time: 3275.1201


In [10]:
sess.run(learning_rate)

0.0081537254

In [8]:
saver.restore(sess, save_dir)